In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

In [ ]:
# download pretrained model and prepared word embedding
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")


In [ ]:
def sentiment_score(inp):

  # tokenize the article name
  tokens = tokenizer.encode(inp,   return_tensors='pt')

  # pass the tokens to model to get sentiment
  output = model(tokens)

  # pass model output logits through a softmax layer
  sentim_scores = torch.nn.functional.softmax(output.logits, dim=-1)

  # returning sum of values that are multiplied by coeficients: 1 for positive, -1 for neative and 0 for neutaral
  # getting in the end following range: numbers closer to 1 means article is most likely positive, closer to -1 most likely negative and closer to 0 is neuteral
  return  sentim_scores.detach().numpy()[0][0]*1+sentim_scores.detach().numpy()[0][1]*-1+sentim_scores.detach().numpy()[0][2]*0

# creating 'sentiment' column in out csv file. Correlating values are output of sentiment_score function with 'Title' as input
merged_df = pd.read_csv("/datasets/output.csv")
merged_df['sentiment'] = merged_df['Title'].apply(lambda x: sentiment_score(x[:150]))
merged_df.to_csv("/datasets/output_sentiment.csv")